In [327]:
import dicom
import numpy as np
import os
import glob
import nibabel as nib

In [350]:
ACDCimgpath='/opt/data/acdc/niftidata/'
ACDClblpath='/opt/output/acdc/norm/1/1L/'

In [351]:
ACDClbls=glob.glob(ACDClblpath)

In [352]:
#Find Patients
results = []
for root, _, files in os.walk(ACDClblpath):
    rootnode = root.split("/")
    label = rootnode[-2]
    tempdir = i.replace('/'+label,'')
    patient = rootnode[-1]
    patient_path = ACDClblpath + patient
    
    results.append(patientVolume(patient_path))
    
    
    #print ('rootnode', rootnode, label, patient, files, tempdir)

patient048_slice8_frame08_label_fix.nii.npy slice8 frame08
patient048_slice4_frame08_label_fix.nii.npy slice4 frame08
patient048_slice7_frame08_label_fix.nii.npy slice7 frame08
patient048_slice2_frame08_label_fix.nii.npy slice2 frame08
patient048_slice1_frame08_label_fix.nii.npy slice1 frame08
patient048_slice7_frame1_label_fix.nii.npy slice7 frame1
patient048_slice6_frame1_label_fix.nii.npy slice6 frame1
patient048_slice4_frame1_label_fix.nii.npy slice4 frame1
patient048_slice5_frame1_label_fix.nii.npy slice5 frame1
patient048_slice1_frame1_label_fix.nii.npy slice1 frame1
patient048_slice3_frame1_label_fix.nii.npy slice3 frame1
patient048_slice2_frame1_label_fix.nii.npy slice2 frame1
patient048_slice8_frame1_label_fix.nii.npy slice8 frame1
patient048_slice5_frame08_label_fix.nii.npy slice5 frame08
patient048_slice6_frame08_label_fix.nii.npy slice6 frame08
patient048_slice3_frame08_label_fix.nii.npy slice3 frame08
patient035_slice5_frame1_label_fix.nii.npy slice5 frame1
patient035_slic

In [358]:
mehtod1[2:5], method3[2:5]

([('patient035', 23.63671875, 76.53515625),
  ('patient042', 551.5234375, 828.28125),
  ('patient100', 12.515625, 281.4609375)],
 [('patient042', 551.5234375, 828.28125),
  ('patient035', 23.63671875, 76.53515625),
  ('patient100', 12.515625, 281.4609375)])

In [332]:
def patientVolume(patient_path):
    for root,_, files in os.walk(patient_path):
        norm = None
        slicedict={}
        #slicenumbers=[]
        for i in files:
            filenode = i.split("_")
            #print (filenode)
            slicenum = filenode[1]
            frame = filenode[2]
            lblpath = root +'/'+ i
        
            #print (i, slicenum, frame)
            origframepath = getNiftiHeader(patient, slicenum, frame)

            origimg = nib.load(origframepath[2])
        
            if norm is not None:
                ps = origimg.header.get('pixdim')[1]
            else: 
                ps = 1
        
            thickness = origimg.header.get('pixdim')[3]
            lbl = np.load(lblpath)
        
            frac = getfrac(lbl)
            area = frac * ps * len(lbl)
            volume = thickness * area
        
            if slicenum in slicedict:
                slicedict[slicenum].update({frame:
                                            {'fraction': frac,
                                             'Area': area, 
                                             'Volume': volume}})
            else:
                slicedict[slicenum] = {frame:
                                    {'fraction': frac,
                                    'Area': area, 
                                    'Volume': volume}}
        
        
        #slicenumbers[slicenum]=([frame, frac, area, volume])
        #print ('rootnode', filenode,slicenum, frame, origframepath, frac, volume)
        #print (slicenum, frame, frac, volume)
        #return frac, area, volume
        ESF, ESV, EDF, EDV = getVolumes(slicedict)
        return (patient, ESV, EDV)

In [318]:
def getVolumes(slicedict):
    ESF = []
    ESVlist = []
    EDF = []
    EDVlist = []

    for key in slicedict:
        frames = list(slicedict[key].keys())
        #print (frames)
        if len(frames) > 2:
            print ('A slice has more than two frames')
        tmp = 0
        for i in range(len(frames)-1):
            #print (slicedict[key][frames[i]]['fraction'], slicedict[key][frames[i+1]]['fraction'] )
            if slicedict[key][frames[i]]['fraction'] < slicedict[key][frames[i+1]]['fraction']:
                ESF.append( key +'_'+ frames[i])
                ESVlist.append(slicedict[key][frames[i]]['Volume'])
                EDF.append(key +'_'+ frames[i+1])
                EDVlist.append(slicedict[key][frames[i+1]]['Volume'])
            else:
                ESF.append(key +'_'+ frames[i+1])
                ESVlist.append(slicedict[key][frames[i+1]]['Volume'])
                EDF.append(key +'_' + frames [i])
                EDVlist.append(slicedict[key][frames[i]]['Volume'])
    ESV = sum(ESVlist)
    EDV = sum(EDVlist)
    return ESF, ESV, EDF, EDV

In [346]:
def getNiftiHeader(patient, slicenum, framenum):
    imgpatientpath = ACDCimgpath+patient
    slicepath = imgpatientpath+'/'+patient+'_'+slicenum
    num = int(framenum.split('frame')[1])
    framepath = slicepath+'/'+patient+'_'+slicenum+'_frame'+str(num)+'.nii'
    return imgpatientpath, slicepath, framepath

In [127]:
def getfrac(img):
    frac = (sum(i == 1 for i in img.flatten()))/float(len(img.flatten()))
    return frac